In [59]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# PostgreSQL connection details
postgres_user = 'postgres'
postgres_password = ''
postgres_host = 'localhost'
postgres_port = '5432'
postgres_db = 'football'

# Create a PostgreSQL connection using SQLAlchemy
postgres_uri = f'postgresql://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_db}'
engine = create_engine(postgres_uri)

# Read the table into a Pandas DataFrame
df_man_city_fixtures=pd.read_sql_table('man_city_fixtures_with_match_id', engine)
df_man_city_shots=pd.read_sql_table('man_city_shots_normalized_with_match_id', engine)
df_haaland_stats = pd.read_sql_table('haaland_statistics', engine)
df_haaland_goals= pd.read_sql_table('haaland_goals', engine)
df_haaland_assists=pd.read_sql_table('haaland_assists',engine)
df_haaland_shots=pd.read_sql_table('haaland_shots', engine)



In [60]:
# Step 1: Merge fixtures and shots
merged_df = pd.merge(
    df_man_city_fixtures,
    df_man_city_shots,
    on=['match_id', 'Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'Result', 'Opponent'],
    how='left'
)

# Step 2: Merge with Haaland stats
merged_df = pd.merge(
    merged_df,
    df_haaland_stats,
    on=['match_id', 'Date', 'Opponent'],
    how='left'
)

# Step 3: Merge with Haaland goals
merged_df = pd.merge(
    merged_df,
    df_haaland_goals,
    on=['match_id', 'Date', 'Venue', 'Comp', 'Result', 'GF', 'GA', 'Opponent'],
    how='left'
)

# Step 4: Handle missing values
merged_df['Minute'] = merged_df['Minute'].fillna(0)
merged_df['Goal_xG'] = merged_df['Goal_xG'].fillna(0)
merged_df = merged_df.fillna('None')



# Step 7: Drop unwanted columns
columns_to_drop = ['Time', 'Unnamed: 0', 'GoalContributionAction', 'ActionType', 'Scorer', 'AssistContributionAction','Squad','Match_xG','Comp','Day','Captain','Goals For','Goals Against','']
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')

# Step 8: save the merged dataframe
merged_df.to_csv('../data/visualizations/merged_df.csv', index=False)



In [51]:
data=merged_df
# Convert date to datetime for better handling
data['Date'] = pd.to_datetime(data['Date'])

# Basic Dataset Information
print(data.info())
print(data.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 59 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              48 non-null     datetime64[ns]
 1   Round             48 non-null     object        
 2   Venue             48 non-null     object        
 3   Result            48 non-null     object        
 4   GF                48 non-null     int64         
 5   GA                48 non-null     int64         
 6   Opponent          48 non-null     object        
 7   xG                48 non-null     float64       
 8   xGA               48 non-null     float64       
 9   Poss              48 non-null     int64         
 10  Formation         48 non-null     object        
 11  Opp Formation     48 non-null     object        
 12  match_id          48 non-null     int64         
 13  Goals             48 non-null     float64       
 14  Shots             48 non-nul

In [70]:
import pandas as pd


# Combine the two DataFrames, aligning the same columns and filling missing values
combined_data = pd.merge(merged_df,
    df_haaland_shots,
    on=['match_id', 'Date','GF','GA'],
    how='left')

combined_data.rename(columns={
    'xG_y' : 'shot_xG'
},inplace=True)

combined_data.drop(columns=['venue'],inplace=True)


# Reset index for clean DataFrame
combined_data.reset_index(drop=True, inplace=True)



# View the combined data
combined_data.to_csv('../data/visualizations/combined_data.csv', index=False)

try:
    # Write data to respective tables (without defining column names explicitly)
    combined_data.to_sql('combined_data', engine, if_exists='replace', index=False)
    

    print("Data successfully inserted into PostgreSQL")

except Exception as e:
    print("Error importing data:", e)


Data successfully inserted into PostgreSQL
